In [1]:
from PIL import Image
import os

In [5]:
dir = "/home/jszumny/Documents/copy of smoke data/newnew/sage-training/obstructed/"

In [6]:
# for filename in os.listdir(dir):
#     if "._" not in filename and ".DS" not in filename:
         
#         image = Image.open(dir + filename)
#         image2 = image.crop((300, 600, 1024, 1500))
#         image3 = image.crop((1024, 600, image.size[0] - 300, 1500))
#         image2.save("/home/jszumny/Documents/copy of smoke data/newnew/sage-training/test/other/" + filename)
#         image3.save("/home/jszumny/Documents/copy of smoke data/newnew/sage-training/test/other/2" + filename)

In [2]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()

use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")

In [3]:
vgg16 = models.vgg16_bn()
vgg16.load_state_dict(torch.load("/home/jszumny/Downloads/vgg16_bn-6c64b313.pth"))
print(vgg16.classifier[6].out_features) # 1000

1000


In [29]:
resume_training = True

if resume_training:
    
    print("Loading pretrained model..")
    vgg16.load_state_dict(torch.load('/home/jszumny/Documents/copy of smoke data/newnew/attempts/3features/fullmodelbest.pt',map_location=torch.device('cpu')))
    print("Loaded!")

Loading pretrained model..
Loaded!


In [30]:
for param in vgg16.features.parameters():
    param.require_grad = False
class_names = ['cloud', 'other', 'smoke']

In [31]:
num_features = vgg16.classifier[6].in_features
features = list(vgg16.classifier.children())[:-1] # Remove last layer
features.extend([nn.Linear(num_features, len(class_names))]) # Add our layer with 4 outputs
vgg16.classifier = nn.Sequential(*features) # Replace the model classifier

In [33]:
from torchvision import transforms
transform = transforms.Compose([            #[1]
 #transforms.Resize(256),                    #[2]
 transforms.CenterCrop(224),                #[3]
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=[0.485, 0.456, 0.406],                #[6]
 std=[0.229, 0.224, 0.225]                  #[7]
 )])



image = Image.open("/home/jszumny/Downloads/try.jpeg")
img_t = transform(image)
batch_t = torch.unsqueeze(img_t, 0)
image.show()


In [34]:
vgg16.eval()
out = vgg16(batch_t)
print(out.shape)

torch.Size([1, 3])


In [35]:
_, index = torch.max(out, 1)
 
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
 
print(class_names[index[0]], percentage[index[0]].item())

other 39.64183807373047


In [36]:
_, indices = torch.sort(out, descending=True)
[(class_names[idx], percentage[idx].item()) for idx in indices[0][:5]]

[('other', 39.64183807373047),
 ('cloud', 32.46512985229492),
 ('smoke', 27.893033981323242)]